In [7]:
%load_ext autoreload
%autoreload 1
# %aimport utils.remote_execute_util
import sys
import os
from dotenv import load_dotenv
from azureml.core import Experiment
from azureml.train.dnn import TensorFlow
from azureml.widgets import RunDetails

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

from amlutils import aml_util as a_util
from amlutils import blobs_util as b_util
load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [8]:
ws = a_util.get_ws()

In [9]:
try:
    ds = b_util.get_ds(ws, 'dsgoethe')
except Exception:
    ds = b_util.register_blob_ws(ws, 'dsgoethe', 'goethe')

In [12]:
compute_target = a_util.prepare_remote_compute(ws, compute_name='gpucluster',
                                               compute_min_nodes=0,
                                               compute_max_nodes=1,
                                               compute_vm_size='STANDARD_NC6')
# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-12-22T21:00:47.982000+00:00', 'errors': None, 'creationTime': '2019-12-22T21:00:45.556653+00:00', 'modifiedTime': '2019-12-22T21:01:01.573093+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount'

In [13]:
script_params = {
    '--data-folder': ds.as_mount(),
    '--output-folder': 'outputs',
    '--batch-size': 50,
    '--first-layer-neurons': 300,
    '--second-layer-neurons': 100,
    '--learning-rate': 0.001
}

estimator = TensorFlow(source_directory='./',
                script_params=script_params,
                compute_target=compute_target,
                entry_script='lm_lstm.py',
                use_gpu=True,
                pip_requirements_file='../requirements_remote_tf.txt',
                framework_version = '1.13'
                      )

In [16]:
experiment = Experiment(workspace=ws, name='expgoethe')
run = experiment.submit(config=estimator)
run

Experiment,Id,Type,Status,Details Page,Docs Page
expgoethe,expgoethe_1577048529_d0935d4b,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


In [17]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSET',…